## A standard suite of tools to work with $SPRUCE$ measurements

- let's get some satellite imagery figured out with `rasterio`
- `astropy` will give appropriate motions of the sun; use the clinometer to get altitude

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import datetime as dt
from scipy import signal, interpolate
import os
import struct
import wave
import rasterio
import rasterio.plot
import rasterio.mask
import fiona
from shapely.geometry import box, LineString, MultiPoint, Polygon
import geopandas as gpd
import pysolar
import datetime as dt
import pytz

import sys
sys.path.append('/home/dax/PythonScripts/3_GITHUB/iyore')
import iyore

%matplotlib inline
    
# audio-processing
import pydub
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
from pydub.playback import play
from pydub.utils import which
AudioSegment.converter = which("ffmpeg")
import crepe # pitch-tracking

# geo-processing
from shapely.geometry import Polygon, asPolygon, asMultiPoint, asPoint
from shapely.affinity import rotate
from shapely.ops import nearest_points

# core + technical
import subprocess 
from sklearn.decomposition import PCA
import os
import datetime as dt
import shutil
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import sys
import numpy as np
from numpy import inf
import pandas as pd


def parse_SPRUCE(meas, start_time, tree):
    
    '''
    Load a raw .CSV file
    '''
    
    SPRUCE = pd.read_csv(meas, header=None) # read in the raw .CSV file
    SPRUCE.columns = ["ch1", "ch2", "ch3", "loop length (ms)"] # give the columns meaningful names
    SPRUCE = SPRUCE.replace(' NULL', np.NaN) # convert 'NULL' strings to numeric NaN
    SPRUCE["loop length (ms)"] = SPRUCE["loop length (ms)"].str[:-1] # strip trailing semicolon
    SPRUCE = SPRUCE.astype('float') # cooerce to floating point numbers

    # a meaningful datetime index
    SPRUCE.index = start_time + (SPRUCE["loop length (ms)"].cumsum()/1000).apply(lambda t: dt.timedelta(seconds=t))
    
    SPRUCE["tree"] = tree
    
    SPRUCE.name = tree

    return SPRUCE

def signal_to_wav(signal, fname, Fs):
    
    """Convert a numpy array into a wav file.

     Args
     ----
     signal : 1-D numpy array
         An array containing the audio signal.
     fname : str
         Name of the audio file where the signal will be saved.
     Fs: int
        Sampling rate of the signal.

    """
    data = struct.pack('<' + ('h'*len(signal)), *signal)
    wav_file = wave.open(fname, 'wb')
    wav_file.setnchannels(1)
    wav_file.setsampwidth(2)
    wav_file.setframerate(Fs)
    wav_file.writeframes(data)
    wav_file.close()
    
    print("complete.")

def match_target_amplitude(sound, target_dBFS):
    
    change_in_dBFS = target_dBFS - sound.dBFS
    
    return sound.apply_gain(change_in_dBFS)

def combine_wav_files(in_path, out_path, out_name):
    
    # routing
    in_path_wav = os.path.join(in_path, "*.wav")
    out_path_name = os.path.join(os.path.join(in_path, out_path), out_name + ".wav")
    
    # input
    paths = glob.glob(in_path_wav)

    stitch = AudioSegment.empty()
    for file in paths:

        sound_file = AudioSegment.from_wav(file)

        stitch+=sound_file
    
    # output
    stitch.export(out_path_name, format="wav")

    
def audio_chunker(audio_path, silence_thresh=-36.0, min_silence=1, len_thresh=200, pad=300):
    
    '''
    Wrapper to load MP3 or WAV files,
    then use `pydub.silence.split_on_silence` to separate 
    
    Inputs
    ------
    audio_path: (str, path) the file you would like to process - can be MP3 or WAV
    
    silence_thresh: (float) an amplitude threshold referenced to decibels full-scale (dBFS)
    
    min_silence: (float) "(in ms) minimum length of a silence to be used for
        a split. default: 1000ms"
        
    len_thresh: (float) ""(in ms or True/False) leave some silence at the beginning
        and end of the chunks. Keeps the sound from sounding like it
        is abruptly cut off.
        When the length of the silence is less than the keep_silence duration
        it is split evenly between the preceding and following non-silent
        segments.
        If True is specified, all the silence is kept, if False none is kept.
        default: 100ms"
        
    pad: (float) 
    
    
    Returns
    -------
    audio_chunks: (list) of `pydub.AudioSegment` objects representing periods of "non-silence"
    
    '''
    
    import pydub
    from pydub import AudioSegment
    from pydub.silence import split_on_silence
    from pydub.utils import mediainfo
    from pydub.playback import play
    from pydub.utils import which

    AudioSegment.converter = which("ffmpeg")
    
    # load the audio with `pydub`
    if(audio_path[-3:] == "wav"):
        sound_file = AudioSegment.from_wav(audio_path)

    elif((audio_path[-3:] == "MP3")|(audio_path[-3:] == "mp3")):
        sound_file = AudioSegment.from_mp3(audio_path)

    else:
        print("there's a pydub generic 'file' version, too, you know...")


    # grab file info while we're at it
    info = mediainfo(audio_path)

    # break it into chunks - the thresholds here might eventually benefit 
    # from some kind of NVSPL preview, I think
    audio_chunks = split_on_silence(sound_file, 
                                    min_silence_len=min_silence, 
                                    silence_thresh=silence_thresh,
                                    keep_silence=pad)
    
    return audio_chunks


def PCA_audio_sampler_prime(audio_path, fft_size=1024, silence_thresh=-36, min_silence=1, len_thresh=20,  
                            pad=300, target_amplitude=-12, compute_pitch=True, verbose=False):
    
    '''
    Making a lathe on which to turn audio.
    The premise here is to divide up the waveform (more or less dynamic) into chunks,
    perform a re-ordination and disperal routine [that is, Principle Component Analysis, PCA]
    and then progress through the dispersed samples using a rotor - however complex
    
    PCA - 
    
    
    '''
    
    audio_chunks = audio_chunker(audio_path, 
                                 silence_thresh=silence_thresh, 
                                 min_silence=min_silence, 
                                 len_thresh=len_thresh, 
                                 pad=pad)
    
    print("Settings produced", len(audio_chunks), "audio chunks")
    
    # grab file info for subsequent computations
    info = mediainfo(audio_path)
    
    # =================================================================
    
    # these are our returns
    normalized_data = []
    indices_out = []
    clip_pitch = []
    chunks_out = []
    
    for i, chunk in enumerate(audio_chunks):
        
        if target_amplitude is not None:
            #amplify the output level to some new value
            chunk = match_target_amplitude(chunk, target_amplitude)
            chunks_out.append(chunk_normalized)

        else:
            chunks_out.append(chunk)
        

        # convert the `pydub` object to `numpy`
        samples = chunk.get_array_of_samples()
        samples = np.array(samples)

        # if stereo, use the L channel only
        if(len(samples.shape) > 1):
            samples = samples[:, 0]

        rate = int(info['sample_rate'])
        #rate = 44100

        # perform a short-time Fourier transform on the data
        frequencies, times, Zxx = signal.stft(samples, 
                                              rate, 
                                              nfft=fft_size)
    
        if(verbose):
            print(Zxx.shape, times.shape)

        if(i%100 == 0):
            print("chunk ", i+1)

        # create the spectrogram by dropping the imaginary (phase) component of the signal
        spectrogram = np.log(np.abs(Zxx))

        if((times.shape[0] > len_thresh)):
        #if((times.shape[0] > len_thresh)&(times.shape[0] < 80)):

            if(verbose):

                plt.figure(figsize=(3, 3))
                plt.pcolormesh(times, frequencies, spectrogram, cmap='magma')
                plt.title("Chunk "+str(i), loc="left")
                plt.show()
                plt.close()

            if(compute_pitch):

                # perform pitch detection
                time, frequency, confidence, activation = crepe.predict(samples, rate, viterbi=False, 
                                                                        step_size=len(samples), verbose=False)

                # if there is reasonable confidence, add it to the attributes
                if(confidence > 0.1):
                    clip_pitch.append(frequency[0])

                else:
                    clip_pitch.append(np.nan)

            else:
                clip_pitch.append(np.nan)

            # then flatten the spectrogram into a vector for PCA
            norm = spectrogram.flatten()/spectrogram.flatten().max()
            normalized_data.append(norm)
            indices_out.append(i)
        
    return chunks_out, normalized_data, indices_out, clip_pitch

def polygon_coords(center, radius, n):
    
    # set up each coordinate
    xs = center[0] + radius * np.cos(np.pi/n * (1 + 2 * np.arange(0, n)))
    ys = center[1] + radius * np.sin(np.pi/n * (1 + 2 * np.arange(0, n)))
    
    coords = np.array([xs, ys]).T
    
    ngon = asPolygon(coords)
    
    return ngon

def gold(value, operator="+", repeats=1):
    
    '''
    Golden ratio calculator; T A P E    M A T H
    
    operator
        "+"  scale up
        "-"  scale down
        
    repeats:  optional
        otherwise, int
    
    '''
    
    big = 1+np.sqrt(5)
    small = 2
    
    
    def up(value):
        
        phi_multiply = small/big - small
        
        value *= -phi_multiply # 1.381966%
        return value
    
    def down(value):
        
        #phi_divide = (small - big)/big
        phi_divide = big/small - small
        value *= -phi_divide # 0.381966%
        return value
    
    def repeat(fn, n):
        
        if n == 1:
            return fn
        else:
            def new_fn(x):
                return fn(repeat(fn, n-1)(x))

            return new_fn
    

    if ((operator == "+")|(operator == 1)|(operator == True)):
        out = repeat(up, repeats)(value)     # scale up
        return out
        
    elif ((operator == "-")|(operator == -1)|(operator == False)):
        out = repeat(down, repeats)(value)      # scale down
        return out
    

### create a masked version of the Alaska 5-meter DEM for this project

In [ ]:
# with rasterio.open("/home/dax/Documents/SPATIAL/Datasets/USGS_AK5M_AK_IFSAR_2010_58.tif") as src:
    
#     # create a rectangular mask
#     mask = gpd.GeoDataFrame([], 
#                         geometry=[box(-148.984514, 63.38777, -148.948851, 63.399623)],
#                         crs="EPSG:4326")
    
#     mask = mask.to_crs(src.crs)
    
#     print(mask.geometry[0])
    
#     out_image, out_transform = rasterio.mask.mask(src, [mask.geometry[0]], crop=True)
#     out_meta = src.meta
    
# out_meta.update({"driver": "GTiff",
#                  "height": out_image.shape[1],
#                  "width": out_image.shape[2],
#                  "transform": out_transform})

# with rasterio.open("/home/dax/Documents/SPATIAL/Datasets/USGS_AK5M_AK_IFSAR_2010_58_Cantwell.tif", "w", **out_meta) as dest:
#     dest.write(out_image)
    

In [ ]:
# %matplotlib inline
# ras = rasterio.open("/home/dax/Documents/SPATIAL/Datasets/USGS_AK5M_AK_IFSAR_2010_58_Cantwell.tif")

# tree = gpd.GeoDataFrame([], geometry=[asPoint([-148.973065, 63.397140])], crs="epsg:4326")
# # tree = tree.to_crs(ras.crs)

# fig, ax = plt.subplots(1, 1, figsize=(10, 10))
# rasterio.plot.show(ras, cmap="YlGn_r", ax=ax)
# tree.plot(ax=ax)
# ax.axis('off')
# plt.show()

# ras.crs

## Load *every* Cantwell Rock measurement

In [ ]:
# SPRUCE = iyore.Dataset(r"/home/dax/Documents/ALBUM/SPRUCE/03 Field Measurements")

# # join all spruce measurements together
# S = pd.concat([parse_SPRUCE(e.path,
#                         dt.datetime(int(e.year), 
#                                     int(e.month), 
#                                     int(e.day), 
#                                     int(e.begin_hour), 
#                                     int(e.begin_minute)),
#                                     e.location + " " + e.num) for e in SPRUCE.meas(location="CantwellRock")])

# # trim off the rather extreme bits
# S.loc[:, "ch1":"ch3"] = S.loc[:, "ch1":"ch3"].mask(S.loc[:, "ch1":"ch3"].abs() > 0.2)

# S

In [ ]:
SPRUCE

### Parse and plot raw voltage measurements from each of 3 leads

In [ ]:
wax = S.loc["2022-08-01 20:00:00":"2022-08-01 20:58:00", :]

plt.plot(wax.index, wax.ch1)
# plt.axvline(dt.datetime(2022, 8, 1, 20, 58), ls="--", color="magenta")
plt.show()

wax.index

# len(wax)

In [ ]:
S.index.min()

In [ ]:
SPRUCE = iyore.Dataset(r"/home/dax/Documents/ALBUM/SPRUCE/03 Field Measurements")

# join all spruce measurements together
S = pd.concat([parse_SPRUCE(e.path,
                        dt.datetime(int(e.year), 
                                    int(e.month), 
                                    int(e.day), 
                                    int(e.begin_hour), 
                                    int(e.begin_minute)),
                                    e.location + " " + e.num) for e in SPRUCE.meas(location="CantwellRock")])

S = S.sort_index()
W = S.dropna()
fig = plt.figure(figsize=(12, 2))
plt.plot(W["loop length (ms)"].cumsum()/1000, W.ch1, ls="", marker="o", ms=1, color="red")
plt.plot(W["loop length (ms)"].cumsum()/1000, W.ch2, ls="", marker="o", ms=1, color="green")
plt.plot(W["loop length (ms)"].cumsum()/1000, W.ch3, ls="", marker="o", ms=1, color="blue")
plt.show()

In [ ]:
np.unique(W.index.date)

In [ ]:
W = W[::100]

fig, ax = plt.subplots(1, 3, figsize=(12, 2))
ax[0].plot(W.ch1, W.ch2, ls="", marker="o", ms=1, color="red")
ax[1].plot(W.ch1, W.ch3, ls="", marker="o", ms=1, color="green")
ax[2].plot(W.ch2, W.ch3, ls="", marker="o", ms=1, color="blue")
plt.show()

In [ ]:
%matplotlib qt

# import matplotlib.dates as dates
# plt_dates = dates.date2num(S.index.to_pydatetime())

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

# ax.scatter(S.cx - S_x0, 
#            S.cy - S_y0, 
#            S.cz - S_z0, 
#            c=fader, alpha=0.1)

p = ax.scatter(S.ch1, 
               S.ch2, 
               S.ch3, 
               s=2,
               c="viridis", alpha=0.1)

# ax.set_xlabel("PC1")
# ax.set_ylabel("PC2")
# ax.set_zlabel("PC3")

# ax.view_init(elev=45, azim=30)

fig.colorbar(p)

plt.show()

In [ ]:
SPRUCE = iyore.Dataset(r"/home/dax/Documents/ALBUM/SPRUCE/03 Field Measurements")

# join all spruce measurements together
S = pd.concat([parse_SPRUCE(e.path,
                        dt.datetime(int(e.year), 
                                    int(e.month), 
                                    int(e.day), 
                                    int(e.begin_hour), 
                                    int(e.begin_minute)),
                                    e.location + " " + e.num) for e in SPRUCE.meas(location="CantwellRock")])


# # S.index = np.arange(len(S))

# trim off the rather extreme bits
S.loc[:, "ch1":"ch3"] = S.loc[:, "ch1":"ch3"].mask(S.loc[:, "ch1":"ch3"].abs() > 0.1)

S = S.resample('30s').quantile(0.5)
# S = S.resample('2min').mean()

print("Spruce data loaded! Geoprocessing...")

tree_lat, tree_long = 63.397140, -148.973065

# defining the timezone
tz = pytz.timezone('US/Alaska')
        
irradiations = []
azimuths = []
for idx in S.index:
    
    print(idx)

    aware_idx = tz.localize(idx.to_pydatetime())
    alt = pysolar.solar.get_altitude(tree_lat, tree_long, aware_idx)
    azi = pysolar.solar.get_azimuth(tree_lat, tree_long, aware_idx)
    irr = pysolar.radiation.get_radiation_direct(aware_idx, alt)

    S.loc[idx, "Azi"] = azi
    S.loc[idx, "Alt"] = alt
    S.loc[idx, "Irr"] = irr
        
# S = S.dropna()
# S.loc[S["Irr"] < 500, "Irr"] = 500

print("Geoprocessing complete!")

In [ ]:
# S

# $x_1 = V_1 \ cos(\phi_1), y_1 = V_1 \ sin(\phi_1), z_1=0$
# $x_2 = V_2 \ cos(\phi_2), y_2 = V_2 \ sin(\phi_2), z_2=0$
# $x_3 = V_3 \ cos(\phi_3), y_3 = V_3 \ sin(\phi_3), z_3=0$
# $x_4 = I \ cos(\phi_{sun}) sin(\alpha_{sun}), y_4 = I \ sin(\phi_{sun})sin(\alpha_{sun}), z_4=I cos(\theta_{sun})$

## see [ellipsoidal coordinates](https://en.wikipedia.org/wiki/Ellipsoidal_coordinates)

# $c_x = (\frac{V_1 cos(\phi_1) + V_2 cos(\phi_2) + V_3 cos(\phi_3) + I cos(\phi_{sun}) sin(\alpha_{sun})}{4})$
# $c_y = (\frac{V_1 sin(\phi_1) + V_2 sin(\phi_2) + V_3 sin(\phi_3) + I sin(\phi_{sun})sin(\alpha_{sun})}{4})$
# $c_z = (\frac{I cos(\alpha_{sun})}{4})$

In [ ]:
np.cos(np.deg2rad(S["Alt"])).min()


In [ ]:
phi = {"ch1":np.deg2rad(285), 
           "ch2":np.deg2rad(12), 
           "ch3":np.deg2rad(140)}

S = S.sort_index()

s_scale = 1e-4

c_x = (S["ch1"]*np.cos(phi["ch1"]) + S["ch2"]*np.cos(phi["ch2"]) +
       S["ch3"]*np.cos(phi["ch3"]) +
       s_scale*S["Irr"]*np.cos(np.deg2rad(S["Azi"]))*np.sin(np.deg2rad(S["Alt"])))/4

c_y = (S["ch1"]*np.sin(phi["ch1"]) + 
       S["ch2"]*np.sin(phi["ch2"]) +              
       S["ch3"]*np.sin(phi["ch3"]) + 
       s_scale*S["Irr"]*np.sin(np.deg2rad(S["Azi"]))*np.sin(np.deg2rad(S["Alt"])))/4

set_screw = 50
c_z = (np.cos(np.deg2rad(S["Alt"])))/(4*set_screw)

S["cx"] = c_x/c_x.max()
S["cy"] = c_y/c_y.max()
S["cz"] = c_z/c_z.max()

# find "zeros" 
# because Principal Components will likely be centered around (0, 0, 0)
S_x0 = (S.cx.max() - S.cx.min())/2
S_y0 = (S.cy.max() - S.cy.min())/2
S_z0 = (S.cz.max() - S.cz.min())/2

S_x0, S_y0, S_z0
fader = (256*S["Irr"]/S["Irr"].max()).values.astype('int')

# -------- DIAGNOSTIC PLOT -----------


%matplotlib qt

import matplotlib.dates as dates
plt_dates = dates.date2num(S.index.to_pydatetime())

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

# ax.scatter(S.cx - S_x0, 
#            S.cy - S_y0, 
#            S.cz - S_z0, 
#            c=fader, alpha=0.1)

p = ax.scatter(S.cx, 
               S.cy, 
               S.cz, 
               s=2,
               c=fader, alpha=0.1)

ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")

# ax.view_init(elev=45, azim=30)

fig.colorbar(p)

plt.show()

## Part Two: PCA

In [ ]:
# audio_path = r"/home/dax/Documents/ALBUM/SPRUCE/Auditization/2022 10 15 SPRUCE samples.wav" # silence -60.0
# audio_path = r"/home/dax/Documents/ALBUM/SPRUCE/Auditization/2022 10 27 SPRUCE woodyard hones.wav" # silence -55.0
# audio_path = r"/home/dax/Documents/ALBUM/SPRUCE/Auditization/2022 10 27 SPRUCE small arms.wav" # silence -47.2
# audio_path = r"/home/dax/Documents/ALBUM/SPRUCE/Auditization/2022 10 01 butt dome compressed clar.wav"
# audio_path = r"/home/dax/Documents/ALBUM/SPRUCE/Auditization/2022 10 15 SPRUCE drumsticks.wav"
# audio_path = r"/media/dax/4/DENAMTVI_2014.wav"
# audio_path = r"/media/dax/4/_KATMDUPO_2016.wav"
# audio_path = r"/media/dax/4/DENAUTOK_2019.wav"
# audio_path = '/home/dax/Documents/ALBUM/bux/2022 09 26 water courses/2022 09 26 water courses.wav'
# audio_path = '/home/dax/Documents/ALBUM/SPRUCE/2022 11 27 risset.wav'
# audio_path = "/home/dax/Documents/ALBUM/SPRUCE/MIDI_bells.wav"
# audio_path = "/home/dax/Documents/ALBUM/SPRUCE/MIDI_chip.wav"
# audio_path = "/home/dax/Documents/ALBUM/SPRUCE/MIDI_voic.wav"
audio_path = "/home/dax/Documents/ALBUM/SPRUCE/MIDI_star.wav"

save_path = os.path.dirname(audio_path) #r"/home/dax/Documents/ALBUM/SPRUCE/04 Audio Fabric"

In [ ]:
save_path

# <font color="salmon">$Tape Math$</font>

In [ ]:
gold(0.36787944231, "-", 1)

# stereo = interpolate.interp1d([0, 180], [-100, 100])
# stereo(12)

S = S.dropna()
S

In [ ]:
4096*2
# 1/2.71828182

In [ ]:
audio_chunks, normalized_data, indices_out, clip_pitch = PCA_audio_sampler_prime(audio_path, 
                                                                   fft_size=2048, 
                                                                   silence_thresh=-40.0, 
                                                                   min_silence=1, # int (ms), should be small at first 
                                                                                   # to let in samples; you can always turn it up
                                                                                 
                                                                   len_thresh=0.05, # we'd like relatively long samples (ex., ms); 
                                                                                  #  this will floor the PCA analysis
                                                                   
                                                                   pad=True,       # this pad should be large (ex. 500 ms), or 'True'
                                                                   target_amplitude=None, # headroom (should be large!)
                                                                   compute_pitch=False, 
                                                                   verbose=False)


### Principal Components Analysis

In [ ]:
sample_lengths = np.array([len(n) for n in normalized_data])

# what percentage do we want to preserve?
preserve = 20 # % ####################################################### MAIN PARAMETER
p = 100 - preserve
shortest_vector = int(np.percentile(sample_lengths, p))

print(shortest_vector)

training_data_raw = []
for sample in normalized_data:
    if len(sample) > shortest_vector:
        training_data_raw.append(sample[:shortest_vector])
        
training_data = np.vstack(training_data_raw)
training_data[training_data == -inf] = 0

pca=PCA(n_components=3)
pca.fit(training_data)

print(pca.explained_variance_ratio_, pca.singular_values_)

projected = pca.fit_transform(training_data)

# ----------  PLOT THE RESULTS  ------------------

fig, ax = plt.subplots(1, 2, figsize=(9,4.8))
ax[0].scatter(projected[:, 0], projected[:, 1], s=1, color="k")
ax[0].set_xlabel('principle component 1', labelpad=15)
ax[0].set_ylabel('principle component 2', labelpad=10)
ax[1].scatter(projected[:, 1], projected[:, 2], s=1, color="k")
ax[1].set_xlabel('principle component 2', labelpad=15)
ax[1].set_ylabel('principle component 3', labelpad=10)

x, y = (0, 0)
ax[0].axhline(y, color="magenta")
ax[0].axvline(x, color="magenta")
ax[1].axhline(y, color="lime")
ax[1].axvline(x, color="lime")

ax[0].set_title("PCA on audio clips", loc="left", fontsize=14, y=1.02)
# plt.savefig(r"C:\Users\DBetchkal\Desktop\PCA_learning.png", dpi=100, bbox_inches="tight")

plt.tight_layout()
plt.show()

In [ ]:
# np.log(S["Irr"]).hist()

wax = S["Irr"]

wax[wax < 500] = 500
np.log10(wax).hist()

### Take $PC_1$, $PC_2$, $PC_3$ and map onto the range of $c_x$, $c_y$, $c_z$

In [ ]:
# select only the 3D coordinates computed above
pathway = S.loc[:, ["cx", "cy", "cz"]].values

# find the ranges of all 3D coordinates
PC1_min, PC2_min, PC3_min = np.nanmin(projected, axis=0)
PC1_max, PC2_max, PC3_max = np.nanmax(projected, axis=0)
x_min, y_min, z_min = np.nanmin(pathway, axis=0)
x_max, y_max, z_max = np.nanmax(pathway, axis=0)

# set up interpolation functions
# for eventual sample selection
map_x = interpolate.interp1d([x_min, x_max],
                             [PC1_min, PC1_max])

map_y = interpolate.interp1d([y_min, y_max],
                             [PC2_min, PC2_max])

map_z = interpolate.interp1d([z_min, z_max],
                             [PC3_min, PC3_max])


# # set up interpolation functions
# # for eventual sample selection
# map_x = interpolate.interp1d([x_min, x_max],
#                              [-25, 40])

# map_y = interpolate.interp1d([y_min, y_max],
#                              [12, -15])

# map_z = interpolate.interp1d([z_min, z_max],
#                              [-12, 14])

how_far_apart = 10 # steps
audio_out = AudioSegment.empty()
sample_number = []
for step in pathway[::how_far_apart, :]:
    
    print("c_x, c_y, c_z:", step)
    
    selector = np.array([map_x(step[0]),
                         map_y(step[1]),
                         map_z(step[2])])
    
    dist_ind = np.sqrt(np.sum((projected-selector)**2, axis=1))
    
    sample_idx = np.argmin(dist_ind)
    sample_number.append(sample_idx)
    print("\twhich corresponds to sample", sample_idx)
    if(sample_idx != 0):
        
        selected_sample = audio_chunks[sample_idx]
        dur = selected_sample.duration_seconds
        sample = selected_sample.fade_in(dur/10).fade_out(dur/5)
        audio_out = audio_out + sample
    

In [ ]:
# S["sample_idx"] = sample_number
# plt.figure(figsize=(14, 1))
# plt.plot(S.index, S["sample_idx"], ls="-", lw=0.5, marker="o", ms=2)
# plt.show()

In [ ]:
# get the datetimes as strings
startdt_out, enddt_out = [dt.datetime.strftime(t, "%Y%m%d_%H%M%S") for t in [S.index.min(), S.index.max()]]

# get the site
site = "SPRUCE"

# version
v = "_3_13_3"

filename = site + " begin " + startdt_out + " end " + enddt_out + " v" + str(v) + ".wav"
audio_out.export(save_path + os.sep + filename, format="wav")

audio_out

In [ ]:
plt.plot(S.index, S["cx"], ls="", marker="o", color="r", ms=2)
plt.plot(S.index, S["cy"], ls="", marker="o", color="g", ms=2)
plt.plot(S.index, S["cz"], ls="", marker="o", color="b", ms=2)
plt.show()

### Wa

In [ ]:
def closest_point_and_distance(p, a, b):
    s = b - a
    w = p - a
    ps = np.dot(w, s)
    if ps <= 0:
        return a, np.linalg.norm(w)
    l2 = np.dot(s, s)
    if ps >= l2:
        closest = b
    else:
        closest = a + ps / l2 * s
    return closest, np.linalg.norm(p - closest)


point = [0, 0, 0]
seg = [[1, 1, 1], [1e3, 1e3, 1e3]] # the point of interest and a much larger value
closest_point_and_distance(point, np.array(seg[0]), np.array(seg[1]))

In [ ]:
def c(S):
    
    phi = {"ch1":np.deg2rad(285), 
           "ch2":np.deg2rad(12), 
           "ch3":np.deg2rad(140)}
    
    for idx, row in S.iterrows():
        
        c_x = 
        c_y = (S["ch1"]*np.sin(phi["ch1"]) + S["ch2"]*np.sin(phi["ch2"]) + S["ch2"]*np.sin(phi["ch2"]) + )/4

In [ ]:
# S = S.dropna()

dir_dict = {2:12, 3:140, 1:285}
color_dict = {1:"red", 2:"green", 3:"blue"}

fig, ax = plt.subplots(3, 1, figsize=(8, 5.5), sharex=True)

for channel in np.arange(3):

    ax[channel].plot(S["Azimuth"], S["ch"+str(channel+1)], ls="", marker="o", ms=1, color=color_dict[channel+1])
    ax[channel].axvline(dir_dict[channel+1], color="magenta", zorder=1)
#     ax[channel].set_ylim(S.loc[:, "ch1":"ch3"].min().min() - 0.05, 
#                          S.loc[:, "ch1":"ch3"].max().max() + 0.05)


plt.xlabel("Sun Azimuth (°)", labelpad=20, fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
wa

In [ ]:
wa = S.dropna()
channel = 1

%matplotlib qt

import matplotlib.dates as dates
plt_dates = dates.date2num(S.index.to_pydatetime())

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.plot(plt_dates, 1000*wa["ch"+str(channel)], wa["Irradiation"], 
        color="k", ls="", ms=1, marker="o", alpha=0.5)

ax.set_xlabel("Sun Azimuth (°)")
ax.set_ylabel("Voltage (mV)")
ax.set_zlabel("Solar Irradiation (W)")

# ax.view_init(elev=45, azim=30)

plt.show()

In [ ]:
print("interior angles", "\n\n", 
      "from 1 to 2:", str((360-285)+12)+"°\n",
      "from 2 to 3:", str(140-12)+"°\n",
      "from 3 to 1:", str(285-140)+"°\n")

print(87+128+145)

In [ ]:
# title = dt.datetime.strftime(S.index.min(), "%Y %m %d") + " " + "ALL"
# title = "ALL MEASUREMENTS TOGETHER"

S = S.loc[S["tree"] == 'CantwellRock 001']

title = "Cantwell Rock TOGETHER"
linesPerSave = 250

fig, ax = plt.subplots(2, 1, figsize=(14, 5), sharex=True)
ax[0].plot(S.index, 1000*S["ch1"], 
           color="red", ls="", marker="o", ms=1, label=dir_dict["ch1"])
ax[0].plot(S.index, 1000*S["ch2"], 
           color="green", ls="", marker="o", ms=1, label=dir_dict["ch2"])
ax[0].plot(S.index, 1000*S["ch3"], 
           color="blue", ls="", marker="o", ms=1, label=dir_dict["ch3"])
ax[0].set_title(title, fontsize=14, loc="left", y=1.05)

ax[0].set_ylabel("Voltage (mV)", labelpad=20, fontsize=12)
ax[0].xaxis.set_major_formatter(mdates.DateFormatter("%m %d\n%H:%M"))

ax[0].axhline(0, ls="--", color="gray", zorder=20)

ax[0].legend(bbox_to_anchor=(1.01, 0.5))
ax[0].set_ylim(-200, 200)

# time_series = time_series.loc[S.index.min():S.index.max()]
# ax[1].plot(time_series.index, time_series["t, 2m"], 
#            color="k", ls="-", label="temp C")

# # draw the 
# for i in np.arange(0, len(S), linesPerSave):
#     ax.axvline(S.iloc[i, :].name, color="red", ls="--", alpha=0.1, zorder=2)

# plt.savefig(r"/home/dax/Documents/ALBUM/Spruce/Field Testing" + os.sep + title + ".png",
#             dpi=150, bbox_inches="tight", facecolor="white")

# ax.set_xlim(S.index.min(), S.index.min()+dt.timedelta(minutes=20))


plt.show()

In [ ]:
help(dt.datetime.replace)

In [ ]:
S.index.max()

In [ ]:
# start_clip = "2022-08-17 00:00:00"
# plt.plot(S.loc[start_clip:, "ch1"], time_series[start_clip:])

S.index = S.index.to_series().apply(lambda t: t.replace(microsecond=0))

J = time_series.merge(S, how="inner", left_index=True, right_index=True)
J

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(1000*J["ch1"], 
         1000*J["t, 2m"], 
         ls="", ms=1, marker="o", alpha=0.2)
plt.show()

In [ ]:
start_clip = "2022-08-17 00:00:00"
plt.figure(figsize=(6, 6))
plt.plot(1000*S.loc[start_clip:, "ch1"], 
         1000*S.loc[start_clip:, "ch2"], 
         ls="", ms=1, marker="o", color="k", alpha=0.2)
plt.show()

In [ ]:
S["Hour"] = S.index.to_series().dt.hour

hourly = np.array([(hr, np.nanpercentile(group.loc[:, ["ch1", "ch2", "ch3"]], 50, axis=0)) for hr, group in S.groupby("Hour")])
hours, medians = hourly.T

channels = pd.DataFrame([list(p) for p in medians], columns=["ch1", "ch2", "ch3"])
plt.plot(channels.index.astype('int'), channels["ch1"], label="ch1", color="red")
plt.plot(channels.index.astype('int'), channels["ch2"], label="ch2", color="green")
plt.plot(channels.index.astype('int'), channels["ch3"], label="ch3", color="blue")
plt.show()

In [ ]:
np.nanpercentile(S.loc[:, ["ch1", "ch2", "ch3"]], 50, axis=0)

In [ ]:
step = 30 # mins
time_steps = np.arange(S.index.min(), S.index.max(), dt.timedelta(minutes=step))


for i, time_step in enumerate(time_steps):
    
    ts = pd.to_datetime(time_step)
    y_max = S.max()[:3].max()
#     y_max = 0.02
#     y_max = 0.1
    y_min = S.min()[:3].min()
#     y_min = 0.0
    
    fig, ax = plt.subplots(3, 1, figsize=(12, 3), sharex=True)
    ax[0].plot(S.loc[ts:ts+dt.timedelta(minutes=step), "ch1"], lw=0.5, color="red")
    ax[1].plot(S.loc[ts:ts+dt.timedelta(minutes=step), "ch2"], lw=0.5, color="green")
    ax[2].plot(S.loc[ts:ts+dt.timedelta(minutes=step), "ch3"], lw=0.5, color="blue")
    
    for a in ax:
        a.set_ylim([y_min, y_max])
        a.xaxis.set_major_formatter(mdates.DateFormatter("%m %d\n%H:%M:%S"))
        
    plt.tight_layout()
    plt.show()
    plt.close()

### A 3D plot

In [ ]:


    print(tree)

In [ ]:
# for creating a responsive plot
# %matplotlib widget

%matplotlib qt

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
step = 50

color_dict = {"BlueHome 001":"blue", "BlueHome 002":"blue", "BlueHome 003":"navy",
              "Brushkana 001":"orange", "Brushkana 003":"salmon",
              "CantwellRock 001":"k", "DenaliHQ 001":"green"}

for tree, obs in S.groupby("tree"):

    ax.plot(obs["ch1"][::step], obs["ch2"][::step], obs["ch3"][::step], 
            color=color_dict[tree], ls="", ms=1, marker="o", alpha=0.5)
ax.view_init(elev=45, azim=30)
plt.show()

In [ ]:
%matplotlib inline

rate = 44100
fft_size = 512

# perform a short-time Fourier transform on the data
frequencies, times, Zxx = signal.stft(S["ch2"], 
                                      rate, 
                                      nfft=fft_size)

spectrogram = np.log(np.abs(Zxx))

plt.figure(figsize=(10, 1))
plt.pcolormesh(times, frequencies, spectrogram, cmap='magma')
plt.title("Spectrogram Representation", loc="left")
plt.show()
plt.close()

In [ ]:
# if(save):
#     print(workingDir+os.sep+titleBar+".wav")

#     signal_to_wav((out_data).astype('int'), 
#                   workingDir+os.sep+titleBar+".wav", 
#                   playback_rate)

playback_rate = 11000
IPython.display.Audio(S["ch3"], rate=playback_rate)

# playback_rate = 10000

# for col in S:
#     if len(col) == 3:
        
#         out_filename = os.path.dirname(meas) + os.sep + os.path.basename(meas)[:-4] + "_" + col + "_audio_" + str(playback_rate) + "Hz.wav"
#         signal_to_wav((10000*(S[col]/S[col].max())).astype('int'),
#                       out_filename,
#                       playback_rate)
